# 기본 설정 및 파일 읽기


In [58]:
import pandas as pd
import numpy as np
import time
import sys


In [5]:
region = ['강원도', '경기도', '경상남도', '경상북도', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전라남도', '전라북도', '제주', '충청남도', '충청북도']
use = ["제공기관유형", "분류체계","확장자","파일이름","파일데이터명","조회수","다운로드수","주기성데이터","등록일","수정일","키워드"]

In [ ]:
#내부망에서 우회하여 파일 다운로드 
for a in region:

  data = pd.read_excel("/content/drive/My Drive/논문_공공데이터포털분석/dataset/{} 200806 통합.xlsx".format(a))

  data.to_excel("{}.xlsx".format(a), encoding='utf-8')
  from google.colab import files
  files.download("{}.xlsx".format(a))
  time.sleep(3)
  print(a, "다운로드 끝")

In [7]:
#파일 드라이브에서 불러오기
def file(name):
    a=name
    name=pd.read_excel('/content/drive/My Drive/논문_공공데이터포털분석/dataset/{} 200806 통합.xlsx'.format(name))
    name=name[use] #필요한 컬럼만
    name['제공지역'] = a #제공지역 컬럼 만들기
    return name

In [ ]:
# groupby로 계산한 데이터프레임 정리하기
# 복합 인덱스의 경우 적용 안돼..
def clean(nana):
  nana = nana[["파일이름"]]
  nana.rename(columns={"파일이름":"count"}, inplace=True)
  nana.sort_values(by="count", ascending=False, inplace = True)
  return nana

In [8]:
#필요 컬럼 솎아내기
gw  =file(region[0])
gg = file(region[1])
gn = file(region[2])
gb = file(region[3])
gj = file(region[4])
dg = file(region[5])
dj = file(region[6])
ps = file(region[7])
sl = file(region[8])
sj = file(region[9])
us = file(region[10])
ic = file(region[11])
jn = file(region[12])
jb = file(region[13])
jj = file(region[14])
cn = file(region[15])
cb = file(region[16])


In [9]:
#전국 단위로 합치기
tot=pd.concat([gw,gg,gn,gb,gj,dg,dj,ps,sl,sj,us,ic,jn,jb,jj,cn,cb])
tot 

#16057행

,제공기관유형,분류체계,확장자,파일이름,파일데이터명,조회수,다운로드수,주기성데이터,등록일,수정일,키워드,제공지역
0,자치행정기관,식품건강,CSV,원주시 일반음식점 정보,2015년 6월 원주시 일반음식점 현황,843,4770,6.0,2019-08-22,2019-08-29,"식당, 맛집, 밥",강원도
1,자치행정기관,공공행정,CSV,강원도_강수량 통계,강원도_강수량 통계_20200803,1171,1122,4.0,2014-10-08,2020-08-03,"통계,강수량,미리미터",강원도
2,자치행정기관,문화관광,XML,강원도 전통 5일장/재래시장,강원도 전통 5일장 안내(xml 제공),478,951,NaN,2014-01-08,2016-06-28,"강원도 전통시장, 5일장, 전통시장",강원도
3,자치행정기관,문화관광,CSV,강원도_음식점현황,강원도_음식점현황_20200729,837,681,NaN,2013-12-07,2020-07-29,"한정식,일식,중국요리",강원도
4,자치행정기관,농축수산,CSV,강원도_인공어초 좌표정보,강원도_인공어초 좌표정보_20200728,179,678,NaN,2018-09-21,2020-07-28,"수산양식,수산자원,해양자원",강원도
...,...,...,...,...,...,...,...,...,...,...,...,...
601,자치행정기관,공공행정,PDF,충청북도 음성군_통계연보,충청북도 음성군_통계연보_20200716,0,1,2.0,2020-05-17,2020-07-16,"인구,면적,소득",충청북도
602,자치행정기관,농축수산,PDF,충청북도_농업미생물 활용 정보,충청북도 농업미생물 활용 정보_2014,95,1,NaN,2020-02-14,2020-02-17,"유기농, 친환경, 유기물",충청북도
603,공공기관,교통물류,CSV,한국도로공사_대전충청권 톨게이트 진출입 일교통량,한국도로공사_대전충청권 톨게이트 진출입 일교통량_20200531,0,0,NaN,2020-07-28,2020-07-28,"대전,충청,교통량",충청북도
604,공공기관,교통물류,CSV,한국도로공사_대전충청권출발 지역간 년누계교통량,한국도로공사_대전충청권출발 지역간 년누계교통량 _20181231,0,0,NaN,2020-07-28,2020-07-28,"대전,충청,교통량",충청북도


# 제공기관별 수량 파악하기

* tot : 지역에 대한 검색어를 입력하여 얻어낸 지역 관련 데이터에 대한 데이터프레임


In [10]:
tot['제공기관유형'].unique()
# 제공기관 유형에는 '자치행정기관', '교육행정기관', '공공기관', '국가행정기관', '교육기관' 이 있습니다.

array(['자치행정기관', '교육행정기관', '공공기관', '국가행정기관', '교육기관'], dtype=object)

In [12]:
tot_pro = tot.groupby(by="제공기관유형").count()
clean(tot_pro)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,count
제공기관유형,
자치행정기관,13901
공공기관,1781
교육행정기관,235
국가행정기관,138
교육기관,2


# 자치행정기관에서 제공한 지자체별 데이터 수량 파악

* tot_region_pre : '제공기관유형'이 '자치행정기관'인 데이터
* tot_region : tot_region_pre에 지역명(시도)을 추가

In [13]:
#지자체것만 빼내기
tot_region_pre = tot[tot['제공기관유형']=="자치행정기관"]
tot_region_pre 

,제공기관유형,분류체계,확장자,파일이름,파일데이터명,조회수,다운로드수,주기성데이터,등록일,수정일,키워드,제공지역
0,자치행정기관,식품건강,CSV,원주시 일반음식점 정보,2015년 6월 원주시 일반음식점 현황,843,4770,6.0,2019-08-22,2019-08-29,"식당, 맛집, 밥",강원도
1,자치행정기관,공공행정,CSV,강원도_강수량 통계,강원도_강수량 통계_20200803,1171,1122,4.0,2014-10-08,2020-08-03,"통계,강수량,미리미터",강원도
2,자치행정기관,문화관광,XML,강원도 전통 5일장/재래시장,강원도 전통 5일장 안내(xml 제공),478,951,NaN,2014-01-08,2016-06-28,"강원도 전통시장, 5일장, 전통시장",강원도
3,자치행정기관,문화관광,CSV,강원도_음식점현황,강원도_음식점현황_20200729,837,681,NaN,2013-12-07,2020-07-29,"한정식,일식,중국요리",강원도
4,자치행정기관,농축수산,CSV,강원도_인공어초 좌표정보,강원도_인공어초 좌표정보_20200728,179,678,NaN,2018-09-21,2020-07-28,"수산양식,수산자원,해양자원",강원도
...,...,...,...,...,...,...,...,...,...,...,...,...
598,자치행정기관,공공행정,XLS,충청북도 청주시_자동차등록현황,충청북도 청주시_자동차등록현황_20200724,99,2,NaN,2013-11-07,2020-07-28,"자동차,건설기계,이륜차",충청북도
599,자치행정기관,식품건강,PDF,충청북도_청소년대상 우리 쌀 활용 음식 정보,충청북도_우리 쌀 맛있는 한그릇_20181008,78,2,NaN,2020-02-14,2020-02-14,"쌀소비촉진, 음식연구, 레시피",충청북도
600,자치행정기관,환경기상,CSV,충청북도 괴산군_사업장폐기물배출자신고사업장현황,충청북도 괴산군_사업장폐기물배출자신고사업장현황_20200722,0,1,NaN,2020-07-22,2020-07-22,"사업장폐기물,폐기물배출자,괴산군",충청북도
601,자치행정기관,공공행정,PDF,충청북도 음성군_통계연보,충청북도 음성군_통계연보_20200716,0,1,2.0,2020-05-17,2020-07-16,"인구,면적,소득",충청북도


In [14]:
tot_region = tot_region_pre #tot_region

In [39]:
#지역별 데이터 수량

tot_region_count = tot_region_pre.groupby('제공지역').count()
tot_region_count = clean(tot_region_count)
tot_region_count


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,count
제공지역,
서울,2631
부산,1652
경기도,1444
전라북도,994
인천,957
경상남도,768
전라남도,757
광주,724
경상북도,705


# 전체 분류체계별 데이터 수량 평균 구하기

지자체에서 제공하는 분류체계별 데이터 수량 평균

* 해당 분류체계의 데이터 수량 합 / 17(지자체 개수)


In [16]:
tot_cat = tot_region.set_index("분류체계").groupby("분류체계").count()
round(clean(tot_cat)/17)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,count
분류체계,
공공행정,161.0
산업고용,96.0
문화관광,90.0
사회복지,76.0
환경기상,75.0
국토관리,58.0
보건의료,52.0
농축수산,48.0
교통물류,46.0


# 지자체별 분류체계별 데이터 수량 구하기

* tot_region_cat : 'tot_region'에서 지자체별, 분류체계별 수량을 구한다

그리고 정렬해서 지자체별로 많이 개방된 분류체계를 알아볼 것임. 유니크한 지점을 찾아내기. 지역 특색을 반영하고 있음.


In [66]:
tot_region_cat = tot_region.set_index("제공지역")
tot_region_cat = tot_region_cat.groupby(["제공지역", "분류체계"]).count()[["파일이름"]]
tot_region_cat.rename(columns={"파일이름" : "count"}, inplace = True)
tot_region_cat

count
제공지역 분류체계       
강원도  공공행정     78
     교육        4
     교통물류     21
     국토관리     21
     농축수산     25
...          ...
충청북도 산업고용     55
     식품건강     20
     재난안전     20
     재정금융     12
     환경기상     41

[244 rows x 1 columns]

In [84]:
#아우 눈아파. txt로 output 빼내기

sys.stdout = open('output_1_지자체별분류체계별_수량.txt','w')


In [85]:
for i in region:
  print(i,": 지자체 분류체계별 데이터 수량 평균","\n","-"*50)
  print(tot_region_cat.loc[i].sort_values(by="count", ascending=False),"\n","-"*50)

# 지자체별 분류체계가 아니라 분류체계별 지자체를 봐보자

잘 안보여...
수량이 평균 이상인 것을 찾아보겠습니다.


In [26]:
tot_cat_region = tot_region.set_index(["분류체계"])
tot_cat_region=tot_cat_region.groupby(["분류체계","제공지역"]).count().loc[:,["제공기관유형"]]
tot_cat_region.rename(columns={"제공기관유형" : "count"}, inplace = True)
tot_cat_region

count
분류체계 제공지역       
공공행정 강원도      78
     경기도     266
     경상남도    140
     경상북도    155
     광주      125
...          ...
환경기상 전라남도     61
     전라북도     68
     제주       21
     충청남도     71
     충청북도     41

[244 rows x 1 columns]

In [86]:
cat = list(tot_cat.index)

sys.stdout = open('output_2_분류체계별 지자체 수량.txt','w')

In [87]:
for i in cat:
  print(i,": 분류체계별 개방 데이터 수량")
  print("*"*10, "평균;", tot_cat.loc[i])
  print(tot_cat_region.loc[i].sort_values(by="count", ascending=False))
  print("-"*40,"\n")

# 지자체별 분류체계별 다운로드수, 조회수

활용도에 대함

* tot_region : 지자체별 데이터프레임 

> 공식 : 해당 지자체의 분류체계별 데이터 다운로드수의 총합 / (지자체의 분류체계 전체 데이터 수량)*(지자체의 분류체계 데이터의 등록 누적일)



In [81]:
#지자체별 분류체계별 조회수, 다운로드수, 주기성데이터 수의 평균을 구한다.
tot_use = round(tot_region.groupby(["제공지역","분류체계"]).mean(),2)
tot_use

조회수   다운로드수  주기성데이터
제공지역 분류체계                        
강원도  공공행정  300.19  117.60    2.55
     교육    202.25   69.25    2.00
     교통물류  112.24   59.05     NaN
     국토관리  187.05   63.71    2.00
     농축수산  260.36  113.00    3.00
...           ...     ...     ...
충청북도 산업고용  187.18  105.15    3.00
     식품건강  234.40   60.80    2.64
     재난안전  154.90   56.50    3.86
     재정금융   95.67   23.50     NaN
     환경기상  158.44   97.00   11.78

[244 rows x 3 columns]

In [88]:
sys.stdout = open('output_3_지자체별분류체계별활용도양상.txt','w')

In [89]:
for i in region:
  print(i,": 지자체별 분류체계별 조회, 다운로드수, 주기성데이터 수 평균")
  print(tot_use.loc[i].sort_values(by="다운로드수", ascending=False))
  print("-"*40,"\n")

## 지자체로 합산 : 평균

In [80]:
a = tot_use.groupby("제공지역").mean().sort_values(by="조회수", ascending=False)
a = pd.merge(tot_region_count, a, on=("제공지역"), how="inner")
a.sort_values(by="count", ascending=False)

,count,조회수,다운로드수,주기성데이터
제공지역,,,,
서울,2631,319.960921,69.035324,0.142433
부산,1652,243.517316,51.622732,0.423676
경기도,1444,187.804936,86.315703,0.493294
전라북도,994,279.894446,84.546906,10.113889
인천,957,208.784555,50.840484,3.144632
경상남도,768,199.313126,88.667480,3.678101
전라남도,757,239.414909,68.358895,2.219267
광주,724,336.301718,71.261762,4.719206
경상북도,705,216.786876,63.994162,3.179121


In [ ]:
a.to_excel("a.xlsx", encoding= "utf-8")
from google.colab import files
files.download("a.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>